# Imports

In [2]:
import numpy as np
import _pickle as pickle
import gym
import time
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras.initializers as initializers
 # CHANGED
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Model

In [3]:
#hello
env = gym.make("Pong-v0")
initializer = initializers.GlorotNormal

x_in = tf.keras.layers.Input(shape = (6400,))
x = tf.keras.layers.Dense(200, kernel_initializer= initializer, activation="relu")(x_in)
x_out = tf.keras.layers.Dense(1, activation="sigmoid")(x)

model = tf.keras.Model(x_in, x_out)

model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["acc"])

In [4]:
# weights = x.get_weights()
# print(weights)

In [5]:
env.observation_space

Box(0, 255, (210, 160, 3), uint8)

In [6]:
env.unwrapped.get_action_meanings()

# NOOP is the same as FIRE (standing still)
# LEFT is the same as LEFTFIRE (down)
# RIGHT is the same as RIGHTFIRE (up)

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [7]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 6400)]            0         
_________________________________________________________________
dense (Dense)                (None, 200)               1280200   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 1,280,401
Trainable params: 1,280,401
Non-trainable params: 0
_________________________________________________________________


# Preprocessing

In [8]:
def prepro(input_frame):
    """ prepro 210x160x3 uint8 frame into 6400 (80x80) 1D float vector """
    input_frame = input_frame[34:194] # crop
    input_frame = input_frame[::2,::2,0] # downsample by factor of 2 (halves the resolution of the image)
    #This takes every other pixel in the image
    input_frame[input_frame == 144] = 0 # erase background (background type 1)
    input_frame[input_frame == 109] = 0 # erase background (background type 2)
    input_frame[input_frame != 0] = 1 # everything else (paddles, ball) just set to 1
    return input_frame.astype(np.float).ravel()

# Discounted Rewards

In [9]:
def discount_rewards(rewards):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(rewards)
    running_add = 0
    for t in reversed(range(0, rewards.size)):
        if rewards[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
        running_add = running_add * gamma + rewards[t]
        discounted_r[t] = running_add
    return discounted_r

# Hyperparamters

In [10]:
render = False
prev_frame = None
game_dimensions = 80*80
gamma = 0.99
resume = False
batch_size = 10

# Main Code

In [16]:
observation = env.reset()
reward_sum = 0
eps = 1
episode_number = 0
running_reward = None
ep_observations, ep_rewards, ep_gradient_log_ps = [], [], []

if resume:
    model.load_weights("ModelWeights")

while True:
    if render:
        env.render()
    
    curr_frame = prepro(observation)
    change_in_frame = curr_frame - prev_frame if prev_frame is not None else np.zeros(game_dimensions)
    prev_frame = curr_frame
    ep_observations.append(change_in_frame)

    # CHANGED 
    # policyforward is happening here 
    with tf.GradientTape(watch_accessed_variables=False) as tape:
        change_in_frame = change_in_frame.reshape((1,6400))
        tape.watch(change_in_frame)
        up_prob = model.predict(change_in_frame)
    # print(up_prob)
    action = 2 if np.random.uniform() < up_prob else 3 # roll the dice!

    # if np.random.random() < eps:
        # action = np.random.randint(2, 4)
    # else:
       # change_in_frame = change_in_frame.reshape((1,6400))
       # up_prob = model.predict(change_in_frame)
        # print(up_prob)
        #if up_prob >= .5:
           # action = 2
       # else:
           # action = 3
    
    observation, reward, done, _ = env.step(action) 
    
    y = 1 if action == 2 else 0
    ep_gradient_log_ps.append(y - up_prob)
    
    # try:
       # ep_gradient_log_ps.append(y - up_prob)
    # except:
       # ep_gradient_log_ps.append(y)
    
    ep_rewards.append(reward)
    reward_sum += reward
    eps -= 0.001
    
    if done:
        comb_ep_observations = np.vstack(ep_observations)
        comb_ep_gradient_log_ps = np.vstack(ep_gradient_log_ps)
        comb_ep_rewards = np.vstack(ep_rewards)
        ep_observations, ep_gradient_log_ps, ep_rewards = [], [], []
        
        discounted_comb_ep_rewards = discount_rewards(comb_ep_rewards)
        discounted_comb_ep_rewards -= np.mean(discounted_comb_ep_rewards) 
        discounted_comb_ep_rewards /= np.std(discounted_comb_ep_rewards)
        
        comb_ep_gradient_log_ps *= discounted_comb_ep_rewards
        grad = tape.gradient(comp_ep_gradient_log_ps, )
        # print(comb_ep_gradient_log_ps)
        
        # model.fit(comb_ep_observations, comb_ep_gradient_log_ps, epochs = 50)
        
        # model.fit(comb_ep_observations, comb_ep_gradient_log_ps, epochs = 50)
        # grad = tape.gradient(comp_ep_gradient_log_ps, comp_ep_hidden_layer_values)
        # print(grad)
        # model.fit()
        # train = tf.train.GradientDescentOptimizer(1.0).minimize(loss)
        # opt = tf.keras.optimizers.Adam(learning_rate=0.1)

         # model.()
        
        running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
        print ('resetting env. episode reward total was %f. running mean: %f' % (reward_sum, running_reward))
        reward_sum = 0
        observation = env.reset() # reset env
        prev_frame = None
        episode_number += 1
        
        if episode_number % batch_size == 0:
            
            # Back Propagation Code Goes Here
            model.save_weights("ModelWeights")
    
    if reward != 0: # Pong has either +1 or -1 reward exactly when game ends.
        print('ep %d: game finished, reward: %f' % (episode_number, reward) + ('' if reward == -1 else ' !!!!!!!!'))

ValueError: Passed in object of type <class 'numpy.ndarray'>, not tf.Tensor